# Kangri Speech Recognition Basic

### Load Dependencies

In [11]:
!pip install librosa
! pip install python_speech_features
import os
import numpy as np
import pandas as pd
from librosa.core import load
from librosa.feature import mfcc
from librosa.util import pad_center
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav

np.random.seed(42)

  Running setup.py bdist_wheel for python-speech-features ... - done
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


### Making DataSet from Audio Files

In [14]:
curr_dir = os.getcwd()
#audio_path = curr_dir + '\Audio'
words = ('Aman1-01.wav','Aman2-01.wav','kiran1-01.wav','kiran2-01.wav')
df = []
for word in words:
#    for clip in os.listdir(curr_dir):
#        file = audio_path + f"\{word}\{clip}"
#        audio,sr = load(word,sr=176400)
#        audio = pad_center(audio,sr,mode='constant')
         (rate,sig) = wav.read(word)
         mfcc_coeff = mfcc(sig,rate)
#        mfcc_coeff = mfcc(audio,sr=sr)
         mfcc_coeff = np.append(mfcc_coeff,word)
         df.append(mfcc_coeff)
df = np.array(df)
data = pd.DataFrame(df,columns=list(range(df.shape[1]-1))+['label'])
data.to_csv('dataset.csv')

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


IndexError: ignored

# Making the Neural Network

### Load Dependencies

In [0]:
from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding
from keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D, Conv2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.optimizers import SGD
from keras.layers import SimpleRNN

Using TensorFlow backend.


### Get the Training and Testing Data and Pre-Processing

In [0]:
audio_features = pd.read_csv('dataset.csv')
X = audio_features.iloc[:,0:audio_features.shape[1]-1]
y = audio_features.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

n_classes = 41
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

(367, 1741)
(92, 1741)
(367,)
(92,)


# Simple Dense Model 

In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(1740,)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(41, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               891392    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
__________

## Performance of Simple Dense Model 

In [0]:
model.fit(X_train, y_train, batch_size=128, epochs=200, verbose=1, validation_data=(X_test, y_test))

Train on 367 samples, validate on 92 samples
Epoch 1/200
367/367 [==============================] - 3s 7ms/step - loss: 5.0187 - acc: 0.0300 - val_loss: 4.0738 - val_acc: 0.1630
Epoch 2/200
367/367 [==============================] - 0s 1ms/step - loss: 4.6760 - acc: 0.0708 - val_loss: 3.3401 - val_acc: 0.2391
Epoch 3/200
367/367 [==============================] - 0s 1ms/step - loss: 3.9192 - acc: 0.1335 - val_loss: 3.0140 - val_acc: 0.2717
Epoch 4/200
367/367 [==============================] - 0s 1ms/step - loss: 3.4722 - acc: 0.1853 - val_loss: 2.7532 - val_acc: 0.3152
Epoch 5/200
367/367 [==============================] - 0s 1ms/step - loss: 3.2049 - acc: 0.2262 - val_loss: 2.5624 - val_acc: 0.3370
Epoch 6/200
367/367 [==============================] - 0s 1ms/step - loss: 2.9761 - acc: 0.2507 - val_loss: 2.3190 - val_acc: 0.3370
Epoch 7/200
367/367 [==============================] - 0s 1ms/step - loss: 2.6762 - acc: 0.2943 - val_loss: 2.2095 - val_acc: 0.4130
Epoch 8/200
367/367 [===